In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
pwd

'/content'

In [ ]:
pip install noisereduce

In [4]:
import os
import torch
list_dir=['Angry','Neutral','Surprise','Sad','Happy']

def get_feature():
  flag=0
  features=0
  labels=0

  for i in range(11,21):
      #input_address='C:/Users/64659/Downloads/Emotional Speech Dataset (ESD)/ESD/'+str(i)+'/'
      # output_address='/content/gdrive/MyDrive/paper4_dataset/ESD_devector/'+str(i)+'/'
      output_address='/content/gdrive/MyDrive/paper4_dataset/ESD_ecapa/'+str(i)+'/'
      for j in list_dir:
          output_dir_tp=output_address + j +'/'

          for file in os.listdir(output_dir_tp):
            #if num>10: break
            tp_input_address=output_dir_tp + file

            tp=torch.load(tp_input_address,map_location=torch.device('cpu'))
            tp=tp.unsqueeze(0)

            tp2 = torch.tensor([int(i-11)], dtype=torch.int64)
            tp2=tp2.unsqueeze(0)
            #print(tp.shape)
            #print(tp2.shape)

            if flag==0:
              flag=1
              features=tp
              labels=tp2
            else:
                features=torch.cat((features,tp), axis=0)
                labels=torch.cat((labels,tp2), axis=0)
  return features, labels

feats,labels=get_feature()
print(feats.shape)
print(labels.shape)

<ipython-input-4-5c9861f012e3>:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  tp=torch.load(tp_input_address,map_location=torch.device('cpu'))


torch.Size([17501, 192])
torch.Size([17501, 1])


In [5]:
labels=labels
data=torch.cat((feats,labels),axis=1)


In [ ]:
print(data.dtype)

torch.float32


In [28]:
import torch.nn as nn
class Network(nn.Module):
    def __init__(self, input_dim=16, output_dim=10, num_domains=2, hidden_dim=384):
        super().__init__()
        layers = []
        layers += [nn.Linear(input_dim, hidden_dim)]
        layers += [nn.ReLU()]
        for _ in range(2):
            layers += [nn.Linear(hidden_dim, hidden_dim)]
            layers += [nn.ReLU()]
        self.shared = nn.Sequential(*layers)

        self.lastlayer = nn.Linear(hidden_dim,output_dim)

        '''
        self.unshared = nn.ModuleList()
        for _ in range(num_domains):
            self.unshared += [nn.Sequential(nn.Linear(hidden_dim, hidden_dim),
                                            nn.ReLU(),
                                            nn.Linear(hidden_dim, hidden_dim),
                                            nn.ReLU(),
                                            nn.Linear(hidden_dim, hidden_dim),
                                            nn.ReLU(),
                                            nn.Linear(hidden_dim, style_dim))]
        '''
    def forward(self, z):
        h = self.shared(z)
        out = self.lastlayer(h)
        return out

    def get_feature(self,z):
        h=self.shared(z)
        return h

In [ ]:
pip install torchinfo

In [ ]:
for param in net.parameters():
  param.requires_grad = False

In [29]:
#from torchinfo import summary

net=Network(input_dim=192,
            hidden_dim=256)
net

Network(
  (shared): Sequential(
    (0): Linear(in_features=192, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=256, bias=True)
    (5): ReLU()
  )
  (lastlayer): Linear(in_features=256, out_features=10, bias=True)
)

In [8]:
summary(net,
        input_size=(1,256),
        col_names=["input_size","output_size","num_params","trainable"],
        col_width=20,
        row_settings=["var_names"])

NameError: name 'summary' is not defined

In [32]:
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import train_test_split

TEST_SIZE = 0.1
BATCH_SIZE = 32
SEED = 42

# generate indices: instead of the actual data we pass in integers instead
train_indices, test_indices, _, _ = train_test_split(
    range(len(data.detach().numpy())),
    data.detach().numpy()[:,-1],
    stratify=data.detach().numpy()[:,-1],
    test_size=TEST_SIZE,
    random_state=SEED
)

# generate subset based on indices
train_split = Subset(data, train_indices)
test_split = Subset(data, test_indices)

# create batches
train_dataloader_simple = DataLoader(train_split, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader_simple = DataLoader(test_split, batch_size=BATCH_SIZE)

In [10]:
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import train_test_split

TEST_SIZE = 0.1
BATCH_SIZE = 32
SEED = 42

# generate indices: instead of the actual data we pass in integers instead
train_indices, test_indices, _, _ = train_test_split(
    range(len(test_dataset)),
    test_dataset[:,-1],
    stratify=test_dataset[:,-1],
    test_size=TEST_SIZE,
    random_state=SEED
)

# generate subset based on indices
train_split = Subset(test_dataset, train_indices)
test_split = Subset(test_dataset, test_indices)

# create batches
train_dataloader_simple = DataLoader(train_split, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader_simple = DataLoader(test_split, batch_size=BATCH_SIZE)

NameError: name 'test_dataset' is not defined

In [ ]:
import numpy as np
print(test_split.indices)
cal=np.zeros((10))

for i in test_split.indices:
  #print(i)
  #print(int(test_split.dataset[i,-1]))
  cal[int(test_split.dataset[i,-1])]+=1

print(cal)


[3674, 9505, 121, 1017, 15602, 2673, 2523, 14560, 5084, 16314, 775, 17466, 840, 3530, 2667, 7495, 13872, 16344, 16546, 8249, 14449, 12138, 10318, 12389, 95, 14115, 17231, 13188, 295, 7861, 1482, 6398, 15264, 15980, 769, 9804, 1396, 9630, 14276, 2620, 2720, 17415, 216, 17198, 1451, 492, 6490, 11696, 11724, 16671, 1763, 4813, 7686, 1215, 8260, 64, 418, 7254, 4545, 13813, 7709, 2727, 15657, 15171, 13654, 11701, 5665, 3970, 16633, 3367, 12265, 15117, 9764, 3325, 14885, 2066, 10765, 10753, 7038, 975, 10480, 14362, 16148, 2434, 4441, 13301, 15707, 7524, 13778, 5422, 2102, 7923, 13760, 9188, 6589, 7545, 16796, 9649, 5087, 5209, 9571, 510, 14707, 10343, 4975, 3071, 16976, 1815, 16380, 13029, 2558, 1570, 7670, 9977, 17479, 10662, 8726, 2785, 3105, 6057, 10722, 12952, 5500, 7004, 16929, 2615, 7255, 1437, 15599, 12955, 2227, 13846, 8980, 16585, 8540, 8656, 14440, 14784, 1130, 5056, 5199, 4989, 3857, 15710, 5186, 15831, 401, 791, 5924, 14559, 13133, 17010, 14833, 5325, 856, 15786, 2083, 8834, 1586

In [ ]:
flag=0
tx=0
for i in test_split.indices:
  if flag==0:
    flag=1
    tx=test_split.dataset[i,:].unsqueeze(0)
    print(tx.shape)
  else:
      tx=torch.cat((tx,test_split.dataset[i,:].unsqueeze(0)), axis=0)

torch.Size([1, 257])


In [ ]:
torch.save(tx, "ESDresnet_baseline_seen1750.pt")

In [33]:
train_dataloader_simple = DataLoader(dataset=train_data_simple,
                                     batch_size=BATCH_SIZE,
                                     shuffle=True,
                                     num_workers=NUM_WORKERS)
test_dataloader_simple = DataLoader(dataset=test_data_simple,
                                    batch_size=BATCH_SIZE,
                                    shuffle=False,
                                    num_workers=NUM_WORKERS)

NameError: name 'train_data_simple' is not defined

In [13]:

# Get a single image batch
image_batch = next(iter(train_dataloader_simple))
image_batch.shape
device=torch.device('cpu')

In [14]:
# Create train_step()
def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer:torch.optim.Optimizer,
               device=device):
  # Put the model in train mode
  model.train()

  # Setup train loss and train accuracy values
  train_loss, train_acc = 0, 0

  # Loop through data loader data batches
  for batch, tp in enumerate(dataloader):
    # Send data to the target device
    X=tp[:,:-1]

    y=tp[:,-1]
    #y=y.type(torch.LongTensor)
    X, y = X.to(device), y.to(device)

    # 1. Forward pass
    y_pred = model(X) # output model logits
    #y_pred = torch.softmax(y_pred,dim=1)
    #print(y_pred,y)
    # 2. Calculate the loss
    y_pred=y_pred.type(torch.float32)
    loss = loss_fn(y_pred, y.long())
    train_loss += loss.item()

    # 3. Optimizer zero grad
    optimizer.zero_grad()

    # 4. Loss backward
    loss.backward()

    # 5. Optimizer step
    optimizer.step()

    # Calculate accuracy metric
    y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
    train_acc += (y_pred_class==y).sum().item()/len(y_pred)

  # Adjust metrics to get average loss and accuracy per batch
  train_loss = train_loss / len(dataloader)
  train_acc = train_acc / len(dataloader)
  return train_loss, train_acc

In [15]:
# Create a test step
def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device=device):
  # Put model in eval mode
  model.eval()

  # Setup test loss and test accuracy values
  test_loss, test_acc = 0,  0

  # Turn on inference mode
  with torch.inference_mode():
    # Loop through DataLoader batches
    for batch, tp in enumerate(dataloader):
      # Send data to the target device
      X=tp[:,:-1]
      y=tp[:,-1]
      X, y = X.to(device), y.to(device)

      # 1. Forward pass
      test_pred_logits = model(X)
      test_pred_logits=test_pred_logits.type(torch.float32)
      # 2. Calculate the loss
      loss = loss_fn(test_pred_logits, y.long())
      test_loss += loss.item()

      # Calculate the accuracy
      test_pred_labels = test_pred_logits.argmax(dim=1)
      test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))

  # Adjust metrics to get average loss and accuracy per batch
  test_loss = test_loss / len(dataloader)
  test_acc = test_acc / len(dataloader)
  return test_loss, test_acc


In [16]:

from tqdm.auto import tqdm

# 1. Create a train function that takes in various model parameters + optimizer + dataloaders + loss function
def train(model: torch.nn.Module,
          train_dataloader,
          test_dataloader,
          optimizer,
          loss_fn: torch.nn.Module = nn.CrossEntropyLoss(),
          epochs: int = 5,
          device=device):

  # 2. Create empty results dictionary
  results = {"train_loss": [],
             "train_acc": [],
             "test_loss": [],
             "test_acc": []}

  # 3. Loop through training and testing steps for a number of epochs
  for epoch in tqdm(range(epochs)):
    train_loss, train_acc = train_step(model=model,
                                       dataloader=train_dataloader,
                                       loss_fn=loss_fn,
                                       optimizer=optimizer,
                                       device=device)
    test_loss, test_acc = test_step(model=model,
                                    dataloader=test_dataloader,
                                    loss_fn=loss_fn,
                                    device=device)

    # 4. Print out what's happening
    print(f"Epoch: {epoch} | Train loss: {train_loss:.4f} | Train acc: {train_acc:.4f} | Test loss: {test_loss:.4f} | Test acc: {test_acc:.4f}")

    # 5. Update results dictionary
    results["train_loss"].append(train_loss)
    results["train_acc"].append(train_acc)
    results["test_loss"].append(test_loss)
    results["test_acc"].append(test_acc)

  # 6. Return the filled results at the end of the epochs
  return results

In [19]:

# Set random seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

# Set number of epochs
NUM_EPOCHS = 400

# Recreate an instance of TinyVGG
'''
model_0 = TinyVGG(input_shape=3, # number of color channels of our target images
                  hidden_units=10,
                  output_shape=len(train_data.classes)).to(device)
'''
model_0=net
# Setup loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model_0.parameters(),
                             lr=0.0001)

# Start the timer
from timeit import default_timer as timer
start_time = timer()

# Train model_0
model_0_results = train(model=model_0,
                        train_dataloader=train_dataloader_simple,
                        test_dataloader=test_dataloader_simple,
                        optimizer=optimizer,
                        loss_fn=loss_fn,
                        epochs=NUM_EPOCHS)

# End the timer and print out how long it took
end_time = timer()
print(f"Total training time: {end_time-start_time:.3f} seconds")


  0%|          | 0/400 [00:00<?, ?it/s]

Epoch: 0 | Train loss: 0.1178 | Train acc: 0.9771 | Test loss: 0.0080 | Test acc: 0.9977
Epoch: 1 | Train loss: 0.0022 | Train acc: 0.9997 | Test loss: 0.0054 | Test acc: 0.9989
Epoch: 2 | Train loss: 0.0005 | Train acc: 1.0000 | Test loss: 0.0057 | Test acc: 0.9989
Epoch: 3 | Train loss: 0.0003 | Train acc: 1.0000 | Test loss: 0.0048 | Test acc: 0.9989
Epoch: 4 | Train loss: 0.0001 | Train acc: 1.0000 | Test loss: 0.0051 | Test acc: 0.9989
Epoch: 5 | Train loss: 0.0001 | Train acc: 1.0000 | Test loss: 0.0046 | Test acc: 0.9989
Epoch: 6 | Train loss: 0.0001 | Train acc: 1.0000 | Test loss: 0.0047 | Test acc: 0.9989
Epoch: 7 | Train loss: 0.0000 | Train acc: 1.0000 | Test loss: 0.0049 | Test acc: 0.9989
Epoch: 8 | Train loss: 0.0000 | Train acc: 1.0000 | Test loss: 0.0047 | Test acc: 0.9989
Epoch: 9 | Train loss: 0.0000 | Train acc: 1.0000 | Test loss: 0.0048 | Test acc: 0.9989
Epoch: 10 | Train loss: 0.0000 | Train acc: 1.0000 | Test loss: 0.0046 | Test acc: 0.9994


KeyboardInterrupt: 

In [20]:
# Create a test step

model_0.eval()

# Setup test loss and test accuracy values
test_loss, test_acc = 0,  0
flag=0
# Turn on inference mode
with torch.inference_mode():
  # Loop through DataLoader batches
  for batch, tp in enumerate(test_dataloader_simple):
    # Send data to the target device
    X=tp[:,:-1]
    y=tp[:,-1]
    X, y = X.to(device), y.to(device)

    # 1. Forward pass
    test_pred_logits = model_0.get_feature(X)
    test_pred_logits=test_pred_logits.type(torch.float32)
    #print(test_pred_logits)
    if flag==0:
      txxx=test_pred_logits
      yxxx=y
      flag=1
    else:
      txxx=torch.cat((txxx,test_pred_logits))
      yxxx=torch.cat((yxxx,y))

yxxx=yxxx.unsqueeze(-1)
output=torch.cat((txxx,yxxx),dim=1)



# Adjust metrics to get average loss and accuracy per batch


In [37]:
# for unseen dataset

model_0.eval()

# Setup test loss and test accuracy values
test_loss, test_acc = 0,  0
flag=0
# Turn on inference mode
with torch.inference_mode():
  # Loop through DataLoader batches
  for batch, tp in enumerate(test_dataloader2):
    # Send data to the target device
    X=tp[:,:-1]
    y=tp[:,-1]
    X, y = X.to(device), y.to(device)
    X=X.type(torch.float32)
    # 1. Forward pass
    test_pred_logits = model_0.get_feature(X)
    test_pred_logits=test_pred_logits.type(torch.float32)
    #print(test_pred_logits)
    if flag==0:
      txxx=test_pred_logits
      yxxx=y
      flag=1
    else:
      txxx=torch.cat((txxx,test_pred_logits))
      yxxx=torch.cat((yxxx,y))

yxxx=yxxx.unsqueeze(-1)
output=torch.cat((txxx,yxxx),dim=1)

In [38]:
torch.save(output, "IEMCAPcapa_ft.pt")

In [ ]:
#%%writefile going_modular/utils.py
"""
File containing various utility functions for PyTorch model training.
"""
import torch

from pathlib import Path

def save_model(model: torch.nn.Module,
               target_dir: str,
               model_name: str):
  """Saves a PyTorch model to a target directory.

  Args:
    model: A target PyTorch model to save.
    target_dir: A directory for saving the model to.
    model_name: A filename for the saved model. Should include
      either ".pth" or ".pt" as the file extension.

  Example usage:
    save_model(model=model_0,
               target_dir="models",
               model_name="05_going_modular_tingvgg_model.pth")
  """
  # Create target directory
  target_dir_path = Path(target_dir)
  target_dir_path.mkdir(parents=True,
                        exist_ok=True)

  # Create model save path
  assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should end with '.pt' or '.pth'"
  model_save_path = target_dir_path / model_name

  # Save the model state_dict()
  print(f"[INFO] Saving model to: {model_save_path}")
  torch.save(obj=model.state_dict(),
             f=model_save_path)

save_model(model_0,"./","ESDrvectorFTmodel.pth")

[INFO] Saving model to: ESDrvectorFTmodel.pth


In [ ]:
# To load in a saved state_dict we have to instantiate a new instance of our model class
loaded_model_0 = net

# Load the saved state_dict of model_0 (this will update the new instance with updated parameters)
loaded_model_0.load_state_dict(torch.load(f="ESDrvectorFTmodel.pth"))

<All keys matched successfully>

In [26]:
import torch
import numpy as np


tp1=0
tp2=0
for i in range(1,6):

  # x1=torch.load('iemdvector_sess' + str(i) +'_neu1.pt',map_location=torch.device('cpu'))
  # x2=np.loadtxt('iemdvector_sess' + str(i) +'_neu1.npy')+2*(i-1)

  x1=torch.load('iemepaca_sess' + str(i) +'_neu1.pt',map_location=torch.device('cpu'))
  x2=np.loadtxt('iemepaca_sess' + str(i) +'_neu1.npy')+2*(i-1)
  print(x2.shape)
  if i==1:
    tp1=x1.detach().numpy()
    tp2=x2
  else:
    tp1=np.concatenate((tp1,x1.detach().numpy()), axis=0)
    tp2=np.concatenate((tp2,x2), axis=0)

print(tp2.shape)

(384,)
(360,)
(319,)
(257,)
(381,)
(1701,)


<ipython-input-26-7ca20c2bb531>:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  x1=torch.load('iemepaca_sess' + str(i) +'_neu1.pt',map_location=torch.device('cpu'))


In [27]:
lis=['ang2','hap','sad','sur']
for j in lis:
  for i in range(1,6):

    x1=torch.load('iemecapa_sess' + str(i) +'_' + j + '.pt',map_location=torch.device('cpu'))
    x2=np.loadtxt('iemecapa_sess' + str(i) +'_' + j + '.npy')+2*(i-1)
    tp1=np.concatenate((tp1,x1.detach().numpy()), axis=0)
    tp2=np.concatenate((tp2,x2), axis=0)
print(tp2.shape)

(3974,)


<ipython-input-27-fa9a1699e49d>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  x1=torch.load('iemecapa_sess' + str(i) +'_' + j + '.pt',map_location=torch.device('cpu'))


In [35]:
tp11=torch.tensor(tp1,dtype=torch.float32)

tp22=torch.from_numpy(tp2)
tp22=torch.tensor(tp22,dtype=torch.float32)

test_dataset=torch.cat((tp11,tp22.unsqueeze(-1)), axis=1)

<ipython-input-35-66d54948cc62>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tp22=torch.tensor(tp22,dtype=torch.float32)


In [36]:
test_dataloader2 = DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [22]:
import torch.nn as nn
class Network_main(nn.Module):
    def __init__(self, input_dim=16, output_dim=64, num_domains=2, hidden_dim=384):
        super().__init__()
        layers = []
        layers += [nn.Linear(input_dim, hidden_dim)]
        layers += [nn.ReLU()]
        for _ in range(2):
            layers += [nn.Linear(hidden_dim, hidden_dim)]
            layers += [nn.ReLU()]
        self.shared = nn.Sequential(*layers)

        self.speakerencoder=nn.ModuleList()
        self.speakerencoder += [nn.Sequential(nn.Linear(hidden_dim, hidden_dim),
                                            nn.ReLU(),
                                            nn.Linear(hidden_dim, hidden_dim),
                                            nn.LayerNorm(),
                                            nn.Linear(hidden_dim, self.out_channels * 2))]

        self.emotionencoder=nn.ModuleList()
        self.emotionencoder += [nn.Sequential(nn.Linear(hidden_dim, hidden_dim),
                                    nn.ReLU(),
                                    nn.Linear(hidden_dim, hidden_dim),
                                    nn.LayerNorm(),
                                    nn.Linear(hidden_dim, self.out_channels * 2))]
        self.decoder=nn.ModuleList()
        self.decoder += [nn.Sequential(nn.Linear(hidden_dim, hidden_dim),
                                    nn.ReLU(),
                                    nn.Linear(hidden_dim, hidden_dim),
                                    nn.ReLU(),
                                    nn.Linear(hidden_dim, input_dim))]

        self.emocls = nn.ModuleList()
        self.emocls += [nn.Sequential(nn.Linear(hidden_dim, hidden_dim),
                                    nn.ReLU(),
                                    nn.Linear(hidden_dim, emoclassnum))]

        self.spkcls = nn.ModuleList()
        self.spkcls += [nn.Sequential(nn.Linear(hidden_dim, hidden_dim),
                                    nn.ReLU(),
                                    nn.Linear(hidden_dim, spkclassnum))]



        '''
        self.unshared = nn.ModuleList()
        for _ in range(num_domains):
            self.unshared += [nn.Sequential(nn.Linear(hidden_dim, hidden_dim),
                                            nn.ReLU(),
                                            nn.Linear(hidden_dim, hidden_dim),
                                            nn.ReLU(),
                                            nn.Linear(hidden_dim, style_dim))]
        '''
    def forward(self, x):
        h=self.shared(x)
        emo_mu, emo_log_std = self.emotionencoder(h)
        spk_mu, spk_log_std = self.speakerencoder(h)

        z_emo = sample(emo_mu, emo_log_std)
        z_spk = sample(spk_mu, spk_log_std)

        dec_in = torch.cat([z_emo, z_spk], dim=1)
        x_hat = self.decoder(dec_in)

        dec_in_emo = torch.cat([emo_mu, emo_log_std], dim=1)
        emoresult=self.emocls(dec_in_emo)

        dec_in_spk = torch.cat([spk_mu, spk_log_std], dim=1)
        spkresult=self.spkcls(dec_in_spk)


        return {'x_hat': x_hat, 'emo_mu': emo_mu, 'emo_log_std': emo_log_std,'z_emo': z_emo,
                 'spk_mu': spk_mu, 'spk_log_std': spk_log_std,"z_spk":z_spk,'emocls':emoresult,'spkcls':spkresult}


    def get_feature(self,z):
        h=self.shared(z)
        return h

    def get_emo(self,x):
        h=self.shared(x)
        emo_mu, emo_log_std = self.emotionencoder(h)
        dec_in = torch.cat([emo_mu, emo_log_std], dim=1)
        emoresult=self.emocls(dec_in)

    def get_spk(self,x):
        h=self.shared(x)
        spk_mu, spk_log_std = self.speakerencoder(h)
        dec_in = torch.cat([spk_mu, spk_log_std], dim=1)
        spkresult=self.spkcls(dec_in)



In [23]:
import torch
import torch.nn as nn

class L1SquaredLoss(nn.Module):
    def __init__(self, reduction='mean'):
        super(L1SquaredLoss, self).__init__()
        self.reduction = reduction

    def forward(self, input, target):
        # Compute the absolute differences
        abs_diff = torch.abs(input - target)
        # Square the differences
        squared_diff = abs_diff ** 2

        if self.reduction == 'mean':
            # Compute the mean of squared differences
            return torch.mean(squared_diff)
        elif self.reduction == 'sum':
            # Compute the sum of squared differences
            return torch.sum(squared_diff)
        else:
            # Return the squared differences as is
            return squared_diff

# Example usage
input = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)
target = torch.tensor([1.5, 2.5, 3.5])

loss_1 = L1SquaredLoss(reduction='mean')
loss = loss_fn(input, target)

print("L1 Squared Loss:", loss.item())

L1 Squared Loss: 8.55704402923584


In [24]:
def sample(mu, logs, std=1.):
    """
    :param mu: [batch, channel]
    :param logs:[batch, channel]
    :param std: sampling std
    :return: sample: [batch, channel]
    """
    assert std > 0.
    eps = torch.normal(torch.zeros_like(mu), torch.ones_like(logs) * std)
    sample = eps * torch.exp(logs) + mu
    return sample

def kl_divergence(mu, log_std):
    """
    :param mu: [B, C]
    :param log_std: [B, C]
    :return:
    """
    post = D.Normal(mu, torch.exp(log_std))
    prior = D.Normal(
        torch.zeros_like(mu, requires_grad=False),
        torch.ones_like(log_std, requires_grad=False))
    kl = D.kl.kl_divergence(post, prior)
    kl = torch.mean(torch.sum(kl, dim=1))
    return kl

def loss_cls(x,y):
    loss_3=nn.CrossEntropyLoss()
    # 1. Forward pass
    x=x.type(torch.float32)
    # 2. Calculate the loss
    loss = loss_3(x, y.long())
    return loss

def loss_fn(self, outputs, x_gt, emo_gt,spk_gt):
    x_hat = outputs['x_hat']
    emo_mu = outputs['emo_mu']
    emo_log_std = outputs['emo_log_std']
    spk_mu = outputs['spk_mu']
    spk_log_std = outputs['spk_log_std']

    emo_kl = kl_divergence(emo_mu, emo_log_std)
    spk_kl = kl_divergence(spk_mu, spk_log_std)

    mi_loss=mutual_information(sample(emo_mu, emo_log_std),sample(spk_mu, spk_log_std))
    loss_2 = nn.MSELoss()
    nll = 0.5*loss_1(x_hat, x_gt) + 0.5*loss_2(x_hat, x_gt)

    emo_cls=loss_cls(emoresult, emo_gt)
    spk_cls=loss_cls(spkresult,spk_gt)
    return nll, emo_kl, spk_kl,mi_loss,

In [ ]:
        if not self.validation: # random scale for robustness
            random_scale = 0.5 + 0.5 * np.random.random()
            wave_tensor = random_scale * wave_tensor

In [25]:
import torch
import numpy as np

def mutual_information(x, y, bins=30):
    # 计算联合直方图
    joint_hist = np.histogram2d(x.numpy(), y.numpy(), bins=bins)[0]
    # 转换为概率
    joint_prob = joint_hist / np.sum(joint_hist)
    # 计算边缘概率
    x_prob = np.sum(joint_prob, axis=1)
    y_prob = np.sum(joint_prob, axis=0)

    # 计算熵
    x_entropy = -np.sum(x_prob * np.log2(x_prob + 1e-9))
    y_entropy = -np.sum(y_prob * np.log2(y_prob + 1e-9))
    joint_entropy = -np.sum(joint_prob * np.log2(joint_prob + 1e-9))

    # 计算互信息
    mi = x_entropy + y_entropy - joint_entropy
    return mi

# 创建两个向量
x = torch.randn(1000)
y = torch.randn(1000)

# 计算互信息
mi = mutual_information(x, y)
print(f"Mutual Information: {mi}")

Mutual Information: 0.35182283009427096


In [ ]:
print(y)

tensor([ 1.2023e-01, -1.0938e-01,  9.2207e-01,  9.7151e-01, -9.9844e-01,
         1.8434e-02,  4.9694e-01,  8.8153e-01, -2.2785e-01, -5.4474e-01,
         1.3488e+00, -2.0486e+00,  1.4976e+00,  3.9248e-01, -3.7313e-01,
         1.1264e+00,  1.0988e+00,  4.9464e-01, -2.2593e-01, -1.8420e-01,
        -1.2816e+00, -1.4587e+00, -7.3192e-01, -1.2724e+00,  2.4385e+00,
        -2.1902e-01,  8.0240e-01,  2.5104e-01,  1.7762e-02,  3.7625e-02,
         1.9639e-02,  8.9844e-01,  1.3591e+00,  1.8403e+00,  3.1363e-01,
        -3.1421e-01,  2.3234e+00,  4.6285e-01,  8.4518e-01,  1.1116e-01,
        -1.0104e-01,  1.7546e+00,  5.3479e-01, -1.4233e+00, -3.7547e-01,
        -9.2472e-01, -2.7425e-01,  3.8717e-01,  6.5675e-01, -7.2609e-01,
         1.2645e+00, -7.1132e-01, -9.8459e-01,  5.5886e-01, -2.2345e-01,
        -8.1661e-02,  5.0329e-01,  3.4073e-01,  6.5831e-01, -9.2395e-01,
         1.0297e+00,  2.9020e-01,  2.9688e+00, -4.8891e-01, -7.4707e-01,
         1.1329e+00,  2.4070e-03, -1.5816e+00, -7.3